In [1]:
import boto3
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)


## conectando ao recurso dynamoDB

In [2]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

table = dynamodb.Table('Movies')



### Criando tabela

In [3]:
title = "The Big New Movie"
year = 2015

### Criando elemento no banco do dynamoDB

In [4]:

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

print("PutItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

PutItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "SQI1AUC2U84612AKPDC9EQJR3FVV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "server": "Server",
            "date": "Fri, 01 Nov 2019 23:25:20 GMT",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "2",
            "connection": "keep-alive",
            "x-amzn-requestid": "SQI1AUC2U84612AKPDC9EQJR3FVV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "2745614147"
        },
        "RetryAttempts": 0
    }
}


### Leitura do elemento inserido no banco

In [5]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print("GetItem succeeded:")
    print(json.dumps(item, indent=4, cls=DecimalEncoder))

GetItem succeeded:
{
    "year": 2015,
    "info": {
        "rating": 0,
        "plot": "Nothing happens at all."
    },
    "title": "The Big New Movie"
}


### Atualização de elemento lido

In [6]:
response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating = :r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(5.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

UpdateItem succeeded:
{
    "Attributes": {
        "info": {
            "actors": [
                "Larry",
                "Moe",
                "Curly"
            ],
            "plot": "Everything happens all at once.",
            "rating": 5.5
        }
    },
    "ResponseMetadata": {
        "RequestId": "1T31UKEFFISVPK1N1PE25HLMSVVV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "server": "Server",
            "date": "Fri, 01 Nov 2019 23:25:27 GMT",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "156",
            "connection": "keep-alive",
            "x-amzn-requestid": "1T31UKEFFISVPK1N1PE25HLMSVVV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "3767510606"
        },
        "RetryAttempts": 0
    }
}


### Consulta de filmes de 1985

In [7]:
from boto3.dynamodb.conditions import Key, Attr
print("Movies from 1985")

response = table.query(
    KeyConditionExpression=Key('year').eq(1985)
)

for i in response['Items']:
    print(i['year'], ":", i['title'])

Movies from 1985


### Consulta de filmes de 2015

In [8]:
from boto3.dynamodb.conditions import Key, Attr
print("Movies from 2015")

response = table.query(
    KeyConditionExpression=Key('year').eq(2015)
)

for i in response['Items']:
    print(i['year'], ":", i['title'])

Movies from 2015
2015 : The Big New Movie


### Apagando tabela de filmes (Opcional)